In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import datetime
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
# import time series csv and SVI csv
case_series_df = pd.read_csv("resources/altered_data/time_series_cases_state.csv", encoding="ISO-8859-1")
SVI_df = pd.read_csv("resources/altered_data/SVI2018_US_COUNTY.csv", encoding="ISO-8859-1")

In [3]:
case_series_df

,UID,FIPS,COUNTY,STATE,Lat,Long_,Combined_Key,3/1/20,3/2/20,3/3/20,...,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21
0,84001001,1001.0,Autauga,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,...,6517,6525,6533,6540,6543,6562,6570,6577,6580,6589
1,84001003,1003.0,Baldwin,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,...,20361,20354,20395,20417,20423,20453,20473,20487,20492,20505
2,84001005,1005.0,Barbour,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",0,0,0,...,2213,2213,2216,2218,2221,2224,2226,2226,2227,2227
3,84001007,1007.0,Bibb,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",0,0,0,...,2529,2530,2535,2534,2535,2535,2536,2536,2537,2542
4,84001009,1009.0,Blount,Alabama,33.982109,-86.567906,"Blount, Alabama, US",0,0,0,...,6387,6388,6402,6408,6415,6420,6424,6426,6443,6444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3108,84056037,56037.0,Sweetwater,Wyoming,41.659439,-108.882788,"Sweetwater, Wyoming, US",0,0,0,...,3947,3951,3953,3961,3990,3990,3990,4002,4006,4022
3109,84056039,56039.0,Teton,Wyoming,43.935225,-110.589080,"Teton, Wyoming, US",0,0,0,...,3530,3532,3547,3549,3562,3562,3562,3593,3593,3609
3110,84056041,56041.0,Uinta,Wyoming,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,0,...,2109,2111,2113,2115,2119,2119,2119,2123,2125,2128
3111,84056043,56043.0,Washakie,Wyoming,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,0,...,889,889,890,890,890,890,890,891,891,891


In [4]:
SVI_df

,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,20044,12398,-999,...,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,23315,21115,8422,...,0,0,0,0,0,3875,508,7.1,0.9,37301
2,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,24222,20600,8220,...,0,0,0,0,0,6303,732,11.0,1.3,40036
3,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,10026,6708,4640,...,0,0,0,0,0,2005,340,10.2,1.7,17280
4,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,53682,45033,20819,...,0,0,0,0,0,10686,796,9.4,0.7,117894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,TEXAS,TX,Duval,48131,"Duval County, Texas",1793.476183,11355,5592,3511,2751,...,1,1,0,2,10,2656,406,24.5,3.7,9592
3138,TEXAS,TX,Zapata,48505,"Zapata County, Texas",998.411980,14369,6388,4405,5609,...,1,0,0,2,10,4206,484,29.3,3.4,10924
3139,TEXAS,TX,Zavala,48507,"Zavala County, Texas",1297.406535,12131,4344,3509,4150,...,1,1,0,2,10,2194,354,18.4,3.0,10672
3140,TEXAS,TX,Dimmit,48127,"Dimmit County, Texas",1328.884075,10663,4408,3309,3148,...,1,1,0,2,11,2345,539,22.2,5.1,8229


In [5]:
#merge on FIPS and then save to a csv file
SVI_case_series_df = pd.merge(case_series_df,SVI_df, on="FIPS", how="outer")
SVI_case_series_df

,UID,FIPS,COUNTY_x,STATE_x,Lat,Long_,Combined_Key,3/1/20,3/2/20,3/3/20,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,84001001.0,1001.0,Autauga,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3875.0,508.0,7.1,0.9,37301.0
1,84001003.0,1003.0,Baldwin,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,20864.0,1646.0,10.2,0.8,195677.0
2,84001005.0,1005.0,Barbour,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,8.0,2558.0,363.0,11.2,1.6,25052.0
3,84001007.0,1007.0,Bibb,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,2.0,1619.0,396.0,7.9,1.9,17696.0
4,84001009.0,1009.0,Blount,Alabama,33.982109,-86.567906,"Blount, Alabama, US",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6303.0,732.0,11.0,1.3,40036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,51640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,5.0,973.0,329.0,15.3,5.2,10411.0
3148,NaN,51710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,3.0,5.0,28169.0,1505.0,12.9,0.7,260366.0
3149,NaN,51720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,1.0,5.0,397.0,140.0,10.1,3.6,6160.0
3150,NaN,51730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,2.0,6.0,4069.0,588.0,13.2,1.9,32249.0


In [6]:
SVI_case_series_df.isna().sum()

UID           39
FIPS           3
COUNTY_x      39
STATE_x       39
Lat           39
              ..
E_UNINSUR     10
M_UNINSUR     10
EP_UNINSUR    10
MP_UNINSUR    10
E_DAYPOP      10
Length: 491, dtype: int64

In [7]:
SVI_case_series_df.to_csv("SVI_case_series_merge.csv")

In [8]:
#load updated SVI_case_series_df
updated_SVI_case_series = pd.read_csv("resources/merged_data/SVI_case_series_merge_updated.csv")
updated_SVI_case_series

,FIPS,COUNTY,ST_ABBR,STATE,Lat,Long_,Combined_Key,3/1/20,3/2/20,3/3/20,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,1001.0,Autauga,AL,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",0,0,0,...,0.0,0.0,0.0,0.0,0.0,3875.0,508.0,7.1,0.9,37301.0
1,1003.0,Baldwin,AL,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",0,0,0,...,0.0,0.0,0.0,1.0,1.0,20864.0,1646.0,10.2,0.8,195677.0
2,1005.0,Barbour,AL,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",0,0,0,...,0.0,0.0,1.0,2.0,8.0,2558.0,363.0,11.2,1.6,25052.0
3,1007.0,Bibb,AL,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",0,0,0,...,0.0,0.0,1.0,2.0,2.0,1619.0,396.0,7.9,1.9,17696.0
4,1009.0,Blount,AL,Alabama,33.982109,-86.567906,"Blount, Alabama, US",0,0,0,...,0.0,0.0,0.0,0.0,0.0,6303.0,732.0,11.0,1.3,40036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,56037.0,Sweetwater,WY,Wyoming,41.659439,-108.882788,"Sweetwater, Wyoming, US",0,0,0,...,0.0,0.0,0.0,0.0,1.0,5240.0,794.0,12.0,1.8,45734.0
3108,56039.0,Teton,WY,Wyoming,43.935225,-110.589080,"Teton, Wyoming, US",0,0,0,...,1.0,0.0,0.0,1.0,2.0,2305.0,603.0,10.0,2.6,29426.0
3109,56041.0,Uinta,WY,Wyoming,41.287818,-110.547578,"Uinta, Wyoming, US",0,0,0,...,0.0,0.0,0.0,0.0,1.0,2499.0,452.0,12.2,2.2,19734.0
3110,56043.0,Washakie,WY,Wyoming,43.904516,-107.680187,"Washakie, Wyoming, US",0,0,0,...,0.0,0.0,0.0,0.0,0.0,1223.0,245.0,15.4,3.1,7859.0


In [9]:
updated_SVI_case_series.isna().sum()

FIPS          3
COUNTY        0
ST_ABBR       9
STATE         0
Lat           0
             ..
E_UNINSUR     9
M_UNINSUR     9
EP_UNINSUR    9
MP_UNINSUR    9
E_DAYPOP      9
Length: 487, dtype: int64

In the future need to go back and find those missing values. For now create new dataframe and put "0" for those null values

In [10]:
SVI_cases_df = updated_SVI_case_series.fillna("0")
SVI_cases_df.isna().sum()

FIPS          0
COUNTY        0
ST_ABBR       0
STATE         0
Lat           0
             ..
E_UNINSUR     0
M_UNINSUR     0
EP_UNINSUR    0
MP_UNINSUR    0
E_DAYPOP      0
Length: 487, dtype: int64

In [11]:
SVI_cases_df.dtypes

FIPS           object
COUNTY         object
ST_ABBR        object
STATE          object
Lat           float64
               ...   
E_UNINSUR      object
M_UNINSUR      object
EP_UNINSUR     object
MP_UNINSUR     object
E_DAYPOP       object
Length: 487, dtype: object


["AREA_SQMI","E_TOTPOP", "E_HU", "E_HH", "E_POV", "E_UNEMP","E_PCI","E_NOHSDP","E_AGE65","E_AGE17", "E_DISABL","E_SNGPNT", "E_MINRTY","E_LIMENG","E_MUNIT","E_MOBILE", "E_CROWD", "E_NOVEH","E_GROUPQ", "EP_POV","EP_UNEMP","EP_PCI","EP_NOHSDP","EP_AGE65", "EP_AGE17", "EP_DISABL", "EP_SNGPNT","EP_MINRTY","EP_LIMENG", "EP_MUNIT", "EP_MOBILE", "EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV","EPL_UNEMP","EPL_PCI","EPL_NOHSDP","SPL_THEME1", "RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","SPL_THEME2","RPL_THEME2","EPL_MINRTY","EPL_LIMENG","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV","F_UNEMP","F_PCI","F_NOHSDP","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_THEME2","F_MINRTY","F_LIMENG","F_THEME3","F_MUNIT	F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_UNINSUR","M_UNINSUR","EP_UNINSUR","MP_UNINSUR","E_DAYPOP"] 

Rows that need to be converted to dtypes


In [12]:
#change dtypes
SVI_cases_df[["AREA_SQMI","E_TOTPOP", "E_HU", "E_HH", "E_POV", "E_UNEMP","E_PCI","E_NOHSDP","E_AGE65","E_AGE17", "E_DISABL","E_SNGPNT", "E_MINRTY","E_LIMENG","E_MUNIT","E_MOBILE", "E_CROWD", "E_NOVEH","E_GROUPQ", "EP_POV","EP_UNEMP","EP_PCI","EP_NOHSDP","EP_AGE65", "EP_AGE17", "EP_DISABL", "EP_SNGPNT","EP_MINRTY","EP_LIMENG", "EP_MUNIT", "EP_MOBILE", "EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV","EPL_UNEMP","EPL_PCI","EPL_NOHSDP","SPL_THEME1", "RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","SPL_THEME2","RPL_THEME2","EPL_MINRTY","EPL_LIMENG","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV","F_UNEMP","F_PCI","F_NOHSDP","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_THEME2","F_MINRTY","F_LIMENG","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_UNINSUR","M_UNINSUR","EP_UNINSUR","MP_UNINSUR","E_DAYPOP"]] = SVI_cases_df[["AREA_SQMI","E_TOTPOP", "E_HU", "E_HH", "E_POV", "E_UNEMP","E_PCI","E_NOHSDP","E_AGE65","E_AGE17", "E_DISABL","E_SNGPNT", "E_MINRTY","E_LIMENG","E_MUNIT","E_MOBILE", "E_CROWD", "E_NOVEH","E_GROUPQ", "EP_POV","EP_UNEMP","EP_PCI","EP_NOHSDP","EP_AGE65", "EP_AGE17", "EP_DISABL", "EP_SNGPNT","EP_MINRTY","EP_LIMENG", "EP_MUNIT", "EP_MOBILE", "EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV","EPL_UNEMP","EPL_PCI","EPL_NOHSDP","SPL_THEME1", "RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","SPL_THEME2","RPL_THEME2","EPL_MINRTY","EPL_LIMENG","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV","F_UNEMP","F_PCI","F_NOHSDP","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_THEME2","F_MINRTY","F_LIMENG","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_UNINSUR","M_UNINSUR","EP_UNINSUR","MP_UNINSUR","E_DAYPOP"]].astype('int')

In [13]:
#check dtypes again
SVI_cases_df.dtypes

FIPS           object
COUNTY         object
ST_ABBR        object
STATE          object
Lat           float64
               ...   
E_UNINSUR       int64
M_UNINSUR       int64
EP_UNINSUR      int64
MP_UNINSUR      int64
E_DAYPOP        int64
Length: 487, dtype: object

In [14]:
#convert the dates to datetime64
SVI_cases_df= SVI_cases_df.dtypes[SVI_cases_df.dtypes =="str"].astype("datetime64")

In [15]:
#read in cases by state only by FIPS
cases_FIPS_df = pd.read_csv("resources/merged_data/ cases_by_date_by_FIPS.csv")
cases_FIPS_df

,DATE,1001,1003,1005,1007,1009,1011,1013,1015,1017,...,24039,24037,24041,24043,24045,24047,25001,25003,25005,25007
0,3/1/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3/2/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3/3/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3/4/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3/5/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,3/27/21,6562,20453,2224,2535,6420,1202,2093,14197,3484,...,2468,5379,1948,13087,7051,3349,11806,5390,59199,0
392,3/28/21,6570,20473,2226,2536,6424,1204,2096,14199,3485,...,2469,5390,1952,13132,7062,3352,11891,5421,59378,0
393,3/29/21,6577,20487,2226,2536,6426,1204,2097,14206,3485,...,2469,5396,1954,13156,7066,3362,11973,5441,59504,0
394,3/30/21,6580,20492,2227,2537,6443,1206,2098,14216,3487,...,2475,5400,1958,13172,7072,3369,12046,5459,59727,0


In [16]:
cases_FIPS_df.dtypes

DATE     object
1001      int64
1003      int64
1005      int64
1007      int64
          ...  
24047     int64
25001     int64
25003     int64
25005     int64
25007     int64
Length: 1225, dtype: object

In [17]:
cases_FIPS_df =cases_FIPS_df[["DATE"]].astype("datetime64")

In [18]:
# make date index 
# cases_FIPS_df=cases_FIPS_df.set_index("DATE")
# cases_FIPS_df

In [19]:
#read SVI data by FIPS and make FIPS the index
SVI_FIPS_df = pd.read_csv("resources/merged_data/SVI_data_by_FIPS.csv")
SVI_FIPS_df=SVI_FIPS_df.set_index("FIPS")
SVI_FIPS_df

,COUNTY,ST_ABBR,STATE,Lat,Long_,Combined_Key,AREA_SQMI,E_TOTPOP,E_HU,E_HH,...,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,Autauga,AL,Alabama,32.539527,-86.644082,"Autauga, Alabama, US",594.443459,55200.0,23315.0,21115.0,...,0.0,0.0,0.0,0.0,0.0,3875.0,508.0,7.1,0.9,37301.0
1003,Baldwin,AL,Alabama,30.727750,-87.722071,"Baldwin, Alabama, US",1589.793007,208107.0,111945.0,78622.0,...,0.0,0.0,0.0,1.0,1.0,20864.0,1646.0,10.2,0.8,195677.0
1005,Barbour,AL,Alabama,31.868263,-85.387129,"Barbour, Alabama, US",885.001636,25782.0,11937.0,9186.0,...,0.0,0.0,1.0,2.0,8.0,2558.0,363.0,11.2,1.6,25052.0
1007,Bibb,AL,Alabama,32.996421,-87.125115,"Bibb, Alabama, US",622.461089,22527.0,9161.0,6840.0,...,0.0,0.0,1.0,2.0,2.0,1619.0,396.0,7.9,1.9,17696.0
1009,Blount,AL,Alabama,33.982109,-86.567906,"Blount, Alabama, US",644.830460,57645.0,24222.0,20600.0,...,0.0,0.0,0.0,0.0,0.0,6303.0,732.0,11.0,1.3,40036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24047,Worcester,MD,Maryland,38.212743,-75.332000,"Worcester, Maryland, US",468.288391,51564.0,56169.0,21672.0,...,0.0,0.0,0.0,1.0,2.0,2986.0,456.0,5.9,0.9,53414.0
25001,Barnstable,MA,Massachusetts,41.729806,-70.288543,"Barnstable, Massachusetts, US",394.228841,213690.0,163181.0,94292.0,...,0.0,0.0,0.0,0.0,1.0,6858.0,730.0,3.2,0.3,202144.0
25003,Berkshire,MA,Massachusetts,42.374284,-73.205248,"Berkshire, Massachusetts, US",926.874879,127328.0,69078.0,55167.0,...,0.0,0.0,0.0,0.0,0.0,3696.0,649.0,3.0,0.5,129162.0


START ML WITH CURRENT DATA (more data will be included in the future)

In [20]:
# # view linear regression graph 
# import multipolyfit as mpf
# from .core import *

# x = np.arange(cases_FIPS_df['DATE'].size)
# y = cases_FIPS_df.loc[:, "3/1/20":"3/31/21"]
# fit = mpf.multipolyfit(x, y, deg=1)
# print ("Slope : " + str(fit[0]))
# print ("Intercept : " + str(fit[1]))

# #Fit function : y = mx + c [linear regression ]
# fit_function = mpf.multipolyfit(fit)

# #Linear regression plot
# plt.plot(cases_FIPS_df['DATE'], fit_function(x))
# #Time series data plot
# plt.plot(cases_FIPS_df['DATE'], cases_FIPS_df.loc[:, "3/1/20":"3/31/21"])

# plt.xlabel('Date')
# plt.ylabel('Cumulative Cases')
# plt.title('Cumulative Cases by Day')
# plt.show()

In [21]:
# Generate our categorical variable lists pd.get_dummies(df['type'])
updated_SVI_case_series= pd.get_dummies(updated_SVI_case_series)
updated_SVI_case_series

,FIPS,Lat,Long_,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,...,"Combined_Key_York, South Carolina, US","Combined_Key_York, Virginia, US","Combined_Key_Young, Texas, US","Combined_Key_Yuba, California, US","Combined_Key_Yukon-Koyukuk, Alaska, US","Combined_Key_Yuma, Arizona, US","Combined_Key_Yuma, Colorado, US","Combined_Key_Zapata, Texas, US","Combined_Key_Zavala, Texas, US","Combined_Key_Ziebach, South Dakota, US"
0,1001.0,32.539527,-86.644082,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1003.0,30.727750,-87.722071,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1005.0,31.868263,-85.387129,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1007.0,32.996421,-87.125115,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1009.0,33.982109,-86.567906,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,56037.0,41.659439,-108.882788,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3108,56039.0,43.935225,-110.589080,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3109,56041.0,41.287818,-110.547578,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3110,56043.0,43.904516,-107.680187,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#check for duplicates rows
updated_SVI_case_series.duplicated().sum()

0

In [36]:
#check for duplicate columns
# updated_SVI_case_series = updated_SVI_case_series[list(updated_SVI_case_series.columns[~updated_SVI_case_series.columns.duplicated()])]
updated_SVI_case_series= updated_SVI_case_series.T.drop_duplicates().T

In [37]:
# Split our preprocessed data into our features and target arrays
y = updated_SVI_case_series.loc[:,  "3/1/20":"3/31/21"].values
X = updated_SVI_case_series.drop(columns=["3/1/20","3/31/21"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
#train the model
fit_model = nn.fit(X_train, y_train, epochs=200)

Epoch 1/200


ValueError: in user code:

    File "/Users/celestemoreno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/celestemoreno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/celestemoreno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/celestemoreno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/celestemoreno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras/engine/training.py", line 949, in compute_loss
        y, y_pred, sample_weight, regularization_losses=s

Try for total deaths


In [25]:
# Split our preprocessed data into our features and target arrays
y = updated_SVI_case_series.loc[:,  "3/1/20"].values
X = updated_SVI_case_series.drop(columns=["3/1/20"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  1655
hidden_nodes_layer2 = 497


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1655)              6754055   
                                                                 
 dense_4 (Dense)             (None, 497)               823032    
                                                                 
 dense_5 (Dense)             (None, 1)                 498       
                                                                 
Total params: 7,577,585
Trainable params: 7,577,585
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  1655
hidden_nodes_layer2 = 497


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1655)              6755710   
                                                                 
 dense_1 (Dense)             (None, 497)               823032    
                                                                 
 dense_2 (Dense)             (None, 1)                 498       
                                                                 
Total params: 7,579,240
Trainable params: 7,579,240
Non-trainable params: 0
_________________________________________________________________


2022-08-03 10:41:17.798580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
#train the model
fit_model = nn.fit(X_train, y_train, epochs=200)

Epoch 1/200
73/73 [==============================] - 2s 17ms/step - loss: nan - accuracy: 0.9807
Epoch 2/200
73/73 [==============================] - 1s 17ms/step - loss: nan - accuracy: 0.9944
Epoch 3/200
73/73 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.9944
Epoch 4/200
73/73 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.9944
Epoch 5/200
73/73 [==============================] - 2s 21ms/step - loss: nan - accuracy: 0.9944
Epoch 6/200
73/73 [==============================] - 2s 22ms/step - loss: nan - accuracy: 0.9944
Epoch 7/200
73/73 [==============================] - 2s 22ms/step - loss: nan - accuracy: 0.9944
Epoch 8/200
73/73 [==============================] - 2s 22ms/step - loss: nan - accuracy: 0.9944
Epoch 9/200
73/73 [==============================] - 2s 22ms/step - loss: nan - accuracy: 0.9944
Epoch 10/200
73/73 [==============================] - 2s 23ms/step - loss: nan - accuracy: 0.9944
Epoch 11/200
73/73 [=========

KeyboardInterrupt: 